In [2]:

# Proposito: Implementacion didactica y practica de forward + backprop (SGD y momentum),
# con soporte para cualquier numero de capas, activaciones 'sigmoide','tanh','relu',
# salida 'binario' (sigmoide) o 'multiclase' (softmax). Todo el codigo esta comentado

# Importar NumPy para operaciones numericas y matriciales
import numpy as np  # biblioteca principal para vectores y matrices

# -------------------------------
# Funciones de activacion y derivadas (cada linea comentada)
# -------------------------------

def sigmoide(z):
    # recibe z (array) y devuelve la funcion sigmoide elemento a elemento
    return 1.0 / (1.0 + np.exp(-z))  # 1/(1+e^-z)


def d_sigmoide(z):
    # calcula la derivada de la sigmoide usando la identidad s'(z)=s(z)*(1-s(z))
    s = sigmoide(z)            # s = sigmoide(z)
    return s * (1.0 - s)       # devolver s*(1-s)


def tanh_act(z):
    # funcion tangente hiperbolica (tanh) aplicada elemento a elemento
    return np.tanh(z)          # numpy implementa tanh vectorizada


def d_tanh(z):
    # derivada de tanh: 1 - tanh(z)^2
    t = np.tanh(z)            # t = tanh(z)
    return 1.0 - t * t        # devolver 1 - t^2


def relu(z):
    # ReLU: devuelve z si z>0, si no devuelve 0
    return np.maximum(0.0, z) # operacion vectorizada, mantiene forma de z


def d_relu(z):
    # derivada de ReLU: 1 para z>0, 0 para z<=0
    return (z > 0).astype(float)  # booleanos convertidos a float (1.0/0.0)


def softmax(z):
    # softmax por filas (cada fila corresponde a un ejemplo)
    # restamos el maximo por fila para estabilidad numerica
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # exponentes estables
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)   # normalizar por fila

# Diccionario que asocia nombres con (func, deriv)
_ACT_FUNCS = {
    'sigmoide': (sigmoide, d_sigmoide),  # para capas ocultas
    'tanh': (tanh_act, d_tanh),
    'relu': (relu, d_relu)
}

# -------------------------------
# Inicializacion de parametros (pesos y biases)
# -------------------------------

def inicializar_parametros(capas, metodo_init='xavier', seed=None):
    # capas: lista de enteros [n_entrada, n_h1, ..., n_salida]
    # metodo_init: 'xavier' o 'he' (usar 'he' con ReLU)
    # seed: semilla aleatoria para reproducibilidad

    # Si el usuario proporciona semilla, fijarla en numpy
    if seed is not None:
        np.random.seed(seed)  # fija la generacion aleatoria

    parametros = {}  # dict que guardara W1,b1,W2,b2,...

    # iterar sobre las transiciones de capa anterior -> capa actual
    for i in range(1, len(capas)):
        n_prev = capas[i-1]  # neuronas en capa anterior
        n_curr = capas[i]    # neuronas en capa actual

        # elegir esquema de inicializacion
        if metodo_init == 'he':
            # He: N(0, 2/n_prev) recomendado para ReLU
            parametros[f'W{i}'] = np.random.randn(n_prev, n_curr) * np.sqrt(2.0 / max(1, n_prev))
        else:
            # Xavier/Glorot aproximado: N(0, 1/n_prev), bueno para sigmoide/tanh
            parametros[f'W{i}'] = np.random.randn(n_prev, n_curr) * np.sqrt(1.0 / max(1, n_prev))

        # biases inicializados en cero (1 x n_curr)
        parametros[f'b{i}'] = np.zeros((1, n_curr))

    # devolver diccionario con todos los parametros inicializados
    return parametros

# -------------------------------
# Forward pass (propagacion hacia adelante)
# -------------------------------

def forward(X, parametros, activaciones_ocultas='relu', problema='multiclase'):
    # X: (m, n_entradas) matriz de ejemplos
    # parametros: diccionario de pesos y biases
    # activaciones_ocultas: string o lista con nombre(s) de activacion para ocultas
    # problema: 'multiclase' o 'binario'

    cache = {}                 # cache para almacenar A0, Z1, A1, ..., ZL, AL
    cache['A0'] = X.copy()     # A0 es la entrada; copiamos para seguridad

    L = len(parametros) // 2   # numero de capas (W,b pares)

    # normalizar activaciones a una lista con longitud L-1 (solo para capas ocultas)
    if isinstance(activaciones_ocultas, str):
        activaciones = [activaciones_ocultas] * max(0, L-1)
    else:
        activaciones = list(activaciones_ocultas)

    # si el usuario dio una lista mas corta o vacia, repetir/rellenar con la primera
    if len(activaciones) != max(0, L-1):
        if len(activaciones) == 0:
            activaciones = ['relu'] * max(0, L-1)
        else:
            activaciones = (activaciones * ((max(0, L-1) // len(activaciones)) + 1))[:max(0, L-1)]

    # recorrer las capas ocultas: 1..L-1
    for i in range(1, L):
        # A_prev: activacion de la capa anterior (m, n_prev)
        A_prev = cache[f'A{i-1}']
        # W y b de la capa i
        W = parametros[f'W{i}']
        b = parametros[f'b{i}']

        # Z = A_prev @ W + b  (producto matricial + sesgo)
        Z = A_prev @ W + b

        # obtener nombre de activacion y la funcion correspondiente
        act_name = activaciones[i-1]
        act_func, _ = _ACT_FUNCS.get(act_name, (relu, d_relu))

        # aplicar la activacion
        A = act_func(Z)

        # guardar Z y A en cache para backprop
        cache[f'Z{i}'] = Z
        cache[f'A{i}'] = A

    # capa de salida: usar logits ZL y aplicar softmax o sigmoide segun problema
    W_L = parametros[f'W{L}']   # pesos de capa final
    b_L = parametros[f'b{L}']   # bias de capa final
    ZL = cache[f'A{L-1}'] @ W_L + b_L  # logits finales (m, n_salida)

    if problema == 'multiclase':
        # softmax para convertir logits a probabilidades por fila
        AL = softmax(ZL)
    else:
        # sigmoide para salida binaria (o multiples salidas binarias)
        AL = sigmoide(ZL)

    # almacenar ZL y AL en cache
    cache[f'Z{L}'] = ZL
    cache[f'A{L}'] = AL

    # devolver cache con todas las activaciones y valores lineales
    return cache

# -------------------------------
# Perdida / Loss
# -------------------------------

def calcular_loss(AL, Y, problema='multiclase'):
    # AL: salidas de la red (m, n_clases) para multiclase o (m,1) para binario
    # Y: etiquetas (one-hot para multiclase; 0/1 para binario)
    m = Y.shape[0]
    eps = 1e-9  # pequeño epsilon para estabilidad en log

    if problema == 'multiclase':
        # cross-entropy: -1/m sum(Y * log(AL))
        loss = -np.sum(Y * np.log(AL + eps)) / m
    else:
        # binary cross-entropy: -1/m sum(y log a + (1-y) log (1-a))
        loss = -np.sum(Y * np.log(AL + eps) + (1 - Y) * np.log(1 - AL + eps)) / m

    return loss

# -------------------------------
# Backpropagation (gradientes y actualizacion)
# -------------------------------

def backprop(cache, parametros, Y, tasa_aprendizaje=0.01, activaciones_ocultas='relu', problema='multiclase', momentum=0.0, vel=None):
    # cache: diccionario con A0,Z1,A1,...
    # parametros: W,b actuales
    # Y: etiquetas correspondientes al batch
    # tasa_aprendizaje: eta
    # activaciones_ocultas: string o lista
    # problema: 'multiclase' o 'binario'
    # momentum: coef. de momentum (0 = sin momentum)
    # vel: diccionario con velocidades previas (si usamos momentum)

    m = Y.shape[0]                # tamaño del batch
    L = len(parametros) // 2      # numero total de capas

    # normalizar activaciones a lista (igual que en forward)
    if isinstance(activaciones_ocultas, str):
        activaciones = [activaciones_ocultas] * max(0, L-1)
    else:
        activaciones = list(activaciones_ocultas)

    if len(activaciones) != max(0, L-1):
        if len(activaciones) == 0:
            activaciones = ['relu'] * max(0, L-1)
        else:
            activaciones = (activaciones * ((max(0, L-1) // len(activaciones)) + 1))[:max(0, L-1)]

    # si usamos momentum y no hay vel proporcionada, inicializar velocidades en cero
    if momentum and vel is None:
        vel = {}
        for i in range(1, L+1):
            vel[f'W{i}'] = np.zeros_like(parametros[f'W{i}'])
            vel[f'b{i}'] = np.zeros_like(parametros[f'b{i}'])

    grads = {}  # diccionario para almacenar gradientes

    # ------------------
    # Gradiente de la capa de salida
    # ------------------
    AL = cache[f'A{L}']  # predicciones del batch

    # para softmax+CE o sigmoid+BCE la expresion dZ = AL - Y es valida
    dZ = AL - Y

    # dW = A_{L-1}^T @ dZ / m
    grads[f'dW{L}'] = cache[f'A{L-1}'].T @ dZ / m
    # db = sum(dZ) / m
    grads[f'db{L}'] = np.sum(dZ, axis=0, keepdims=True) / m

    # ------------------
    # Retropropagar por capas ocultas en orden inverso
    # ------------------
    for i in range(L-1, 0, -1):
        # dA_prev: gradiente sobre activacion de la capa anterior
        dA_prev = dZ @ parametros[f'W{i+1}'].T
        # recuperar Z de la capa actual para calcular derivada
        Z_curr = cache[f'Z{i}']

        # escoger derivada segun activacion
        act_name = activaciones[i-1]
        _, deriv = _ACT_FUNCS.get(act_name, (relu, d_relu))

        # dZ = dA_prev * f'(Z)
        dZ = dA_prev * deriv(Z_curr)

        # calcular dW y db para la capa i
        grads[f'dW{i}'] = cache[f'A{i-1}'].T @ dZ / m
        grads[f'db{i}'] = np.sum(dZ, axis=0, keepdims=True) / m

    # ------------------
    # Actualizar parametros: SGD simple o con momentum
    # ------------------
    for i in range(1, L+1):
        if momentum and vel is not None:
            # actualizar velocidad: v = momentum * v + (1-momentum) * grad
            vel[f'W{i}'] = momentum * vel[f'W{i}'] + (1.0 - momentum) * grads[f'dW{i}']
            vel[f'b{i}'] = momentum * vel[f'b{i}'] + (1.0 - momentum) * grads[f'db{i}']

            # actualizar parametros usando la velocidad
            parametros[f'W{i}'] -= tasa_aprendizaje * vel[f'W{i}']
            parametros[f'b{i}'] -= tasa_aprendizaje * vel[f'b{i}']
        else:
            # SGD estandar: w = w - eta * grad
            parametros[f'W{i}'] -= tasa_aprendizaje * grads[f'dW{i}']
            parametros[f'b{i}'] -= tasa_aprendizaje * grads[f'db{i}']

    # devolver parametros actualizados y objeto vel (para seguir usando momentum)
    return parametros, vel

# -------------------------------
# Entrenamiento (funcion principal)
# -------------------------------

def entrenar(X, Y, capas, problema='multiclase', activaciones_ocultas='relu', metodo_init='xavier',
             tasa_aprendizaje=0.01, epocas=1000, batch_size=None, verbose=True, seed=None, momentum=0.0):
    # X: entradas (m, n_entradas)
    # Y: etiquetas (one-hot para multicase, 0/1 para binario)
    # capas: lista de tamaños por capa
    # problema: 'multiclase' o 'binario'
    # activaciones_ocultas: 'sigmoide'|'tanh'|'relu' o lista equivalente
    # metodo_init: 'xavier' o 'he'
    # tasa_aprendizaje: eta
    # epocas: numero de iteraciones completas sobre el dataset
    # batch_size: None -> batch completo; entero -> mini-batch
    # verbose: imprime progreso (loss)
    # seed: semilla para reproducibilidad
    # momentum: coeficiente 0..<1 para usar momentum

    # inicializar parametros con el metodo elegido
    parametros = inicializar_parametros(capas, metodo_init, seed)
    m = X.shape[0]  # numero de ejemplos

    # si no se da batch_size o es invalido, usar batch completo
    if batch_size is None or batch_size <= 0:
        batch_size = m

    vel = None  # variable para almacenar velocidades si se usa momentum

    # bucle principal de entrenamiento por epocas
    for ep in range(1, epocas + 1):
        # permutar los datos aleatoriamente cada epoca
        indices = np.random.permutation(m)
        X_shuf = X[indices]
        Y_shuf = Y[indices]

        # iterar sobre mini-batches
        for start in range(0, m, batch_size):
            end = start + batch_size
            X_batch = X_shuf[start:end]
            Y_batch = Y_shuf[start:end]

            # forward sobre el batch
            cache = forward(X_batch, parametros, activaciones_ocultas, problema)

            # backprop sobre el batch y actualizar parametros
            parametros, vel = backprop(cache, parametros, Y_batch, tasa_aprendizaje, activaciones_ocultas, problema, momentum, vel)

        # si verbose, calcular y mostrar loss en todo el set cada cierto numero de epocas
        if verbose and (ep == 1 or ep % max(1, epocas // 10) == 0):
            cache_full = forward(X, parametros, activaciones_ocultas, problema)
            loss = calcular_loss(cache_full[f'A{len(capas)-1}'], Y, problema)
            print(f"Época {ep}/{epocas} - Loss: {loss:.6f}")

    # devolver parametros finales entrenados
    return parametros

# -------------------------------
# Prediccion (uso despues de entrenar)
# -------------------------------

def predecir(X, parametros, problema='multiclase', activaciones_ocultas='relu'):
    # realiza forward completo y devuelve clases y probabilidades
    cache = forward(X, parametros, activaciones_ocultas, problema)  # forward
    AL = cache[f'A{len(parametros)//2}']  # salida final

    if problema == 'multiclase':
        # devolver indice de la clase y la matriz de probabilidades
        clases = np.argmax(AL, axis=1)
        probs = AL
    else:
        # si salida es un solo valor por ejemplo, aplicar umbral 0.5
        if AL.shape[1] == 1:
            clases = (AL[:, 0] >= 0.5).astype(int)
            probs = AL[:, 0:1]
        else:
            # multiples salidas binarias
            clases = (AL >= 0.5).astype(int)
            probs = AL

    return clases, probs

# -------------------------------
# EJEMPLO: XOR y multiclasico sintetico
# -------------------------------

if __name__ == '__main__':
    # -- Ejemplo XOR (problema binario) --
    # crear entradas y salidas para XOR
    X_xor = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=float)  # 4x2
    Y_xor = np.array([[0],[1],[1],[0]], dtype=float)          # 4x1

    # definir arquitectura: entrada 2, una oculta con 4 neuronas, salida 1
    capas_xor = [2, 4, 1]

    # entrenar la red para XOR: usando tanh en ocultas (buena para XOR), alta lr
    parametros_xor = entrenar(X_xor, Y_xor, capas_xor, problema='binario', activaciones_ocultas='tanh',
                              metodo_init='xavier', tasa_aprendizaje=0.5, epocas=8000, batch_size=None, verbose=True, seed=42)

    # predecir y mostrar resultados para XOR
    clases_xor, probs_xor = predecir(X_xor, parametros_xor, problema='binario', activaciones_ocultas='tanh')
    print('XOR - Predicciones:', clases_xor)
    print('XOR - Probabilidades:', probs_xor)

    # ---- PRUEBA MANUAL XOR ----
    nuevo = np.array([[1, 1]])   # entrada a probar
    clase, prob = predecir(nuevo, parametros_xor, problema='binario', activaciones_ocultas='tanh')

    print("Entrada:", nuevo)
    print("Clase predicha:", clase)
    print("Probabilidad:", prob)

    # -- Ejemplo multiclase sintetico --
    np.random.seed(0)            # semilla para reproducibilidad
    N = 200                     # numero de ejemplos
    X_mc = np.random.randn(N, 2)  # ejemplo con 2 features
    Y_idx = np.random.randint(0, 3, size=(N,))  # etiquetas 0..2
    Y_mc = np.zeros((N, 3))      # convertimos a one-hot
    Y_mc[np.arange(N), Y_idx] = 1

    # arquitectura: 2 entradas, 2 capas ocultas, 3 salidas
    capas_mc = [2, 16, 8, 3]

    # entrenar con ReLU en la primera oculta y tanh en la segunda
    parametros_mc = entrenar(X_mc, Y_mc, capas_mc, problema='multiclase', activaciones_ocultas=['relu','tanh'],
                              metodo_init='he', tasa_aprendizaje=0.01, epocas=300, batch_size=32, verbose=True, seed=1)

    # predecir y mostrar primeras 10 predicciones
    clases_mc, probs_mc = predecir(X_mc, parametros_mc, problema='multiclase', activaciones_ocultas=['relu','tanh'])
    print('Multiclase - primeros 10 predicciones:', clases_mc[:10])

# FIN del archivo


Época 1/8000 - Loss: 0.716595
Época 800/8000 - Loss: 0.007541
Época 1600/8000 - Loss: 0.003307
Época 2400/8000 - Loss: 0.002112
Época 3200/8000 - Loss: 0.001551
Época 4000/8000 - Loss: 0.001225
Época 4800/8000 - Loss: 0.001012
Época 5600/8000 - Loss: 0.000862
Época 6400/8000 - Loss: 0.000751
Época 7200/8000 - Loss: 0.000665
Época 8000/8000 - Loss: 0.000597
XOR - Predicciones: [0 1 1 0]
XOR - Probabilidades: [[4.24916120e-05]
 [9.99333402e-01]
 [9.99325033e-01]
 [1.00203049e-03]]
Entrada: [[1 1]]
Clase predicha: [0]
Probabilidad: [[0.00100203]]
Época 1/300 - Loss: 1.305247
Época 30/300 - Loss: 1.098291
Época 60/300 - Loss: 1.077576
Época 90/300 - Loss: 1.068970
Época 120/300 - Loss: 1.063496
Época 150/300 - Loss: 1.059181
Época 180/300 - Loss: 1.055707
Época 210/300 - Loss: 1.052893
Época 240/300 - Loss: 1.050453
Época 270/300 - Loss: 1.047897
Época 300/300 - Loss: 1.045593
Multiclase - primeros 10 predicciones: [0 2 2 2 2 2 2 1 2 2]
